<a href="https://colab.research.google.com/github/jumbokh/nknu-class/blob/main/CNN/notebooks/06_05_Data_Augmentation_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST 模型強化

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -fs /content/gdrive/My\ Drive /app

Mounted at /content/gdrive


In [2]:
!tar -xzvf /app/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# 檢查是否安裝成功
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

cuda/include/cudnn.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.7
cuda/lib64/libcudnn.so.7.5.0
cuda/lib64/libcudnn_static.a
#define CUDNN_MAJOR 7
#define CUDNN_MINOR 5
#define CUDNN_PATCHLEVEL 0
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#include "driver_types.h"


## 步驟1：載入 MNIST 手寫阿拉伯數字資料

In [3]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

# 載入 MNIST 手寫阿拉伯數字資料
(x_train, y_train),(x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


## 步驟2：改用 CNN 模型

In [4]:
# 建立模型
from tensorflow.keras import layers
import numpy as np

input_shape=(28, 28, 1)
# 增加一維在最後面
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# CNN 模型
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation="softmax"),
    ]
)

# 設定優化器(optimizer)、損失函數(loss)、效能衡量指標(metrics)的類別
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 步驟3：資料增補(Data Augmentation)

In [5]:
# 參數設定
batch_size = 1000
epochs = 5

# 資料增補定義
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,         # 特徵縮放
        rotation_range=10,      # 旋轉 10 度
        zoom_range=0.1,         # 拉遠/拉近 10%
        width_shift_range=0.1,  # 寬度偏移  10%
        height_shift_range=0.1) # 高度偏移  10%

# 增補資料，進行模型訓練
datagen.fit(x_train)
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
          validation_data=datagen.flow(x_test, y_test, batch_size=batch_size), verbose=2,
          steps_per_epoch=x_train.shape[0]//batch_size)

Epoch 1/5
60/60 - 54s - loss: 1.3753 - accuracy: 0.5472 - val_loss: 0.5866 - val_accuracy: 0.8311
Epoch 2/5
60/60 - 25s - loss: 0.5452 - accuracy: 0.8286 - val_loss: 0.3324 - val_accuracy: 0.9027
Epoch 3/5
60/60 - 23s - loss: 0.3750 - accuracy: 0.8837 - val_loss: 0.2350 - val_accuracy: 0.9322
Epoch 4/5
60/60 - 23s - loss: 0.3115 - accuracy: 0.9041 - val_loss: 0.1902 - val_accuracy: 0.9448
Epoch 5/5
60/60 - 23s - loss: 0.2698 - accuracy: 0.9188 - val_loss: 0.1696 - val_accuracy: 0.9508


In [6]:
# 評分(Score Model)
score=model.evaluate(x_test, y_test, verbose=0)

for i, x in enumerate(score):
    print(f'{model.metrics_names[i]}: {score[i]:.4f}')

loss: 10.0298
accuracy: 0.9759


## 步驟4：測試自行繪製的數字

In [8]:
# 使用小畫家，繪製 0~9，實際測試看看
from skimage import io
from skimage.transform import resize
import numpy as np

# 讀取影像並轉為單色
uploaded_file = '/app/DL_Book/myDigits/9.png'
image1 = io.imread(uploaded_file, as_gray=True)

# 縮為 (28, 28) 大小的影像
image_resized = resize(image1, (28, 28), anti_aliasing=True)    
X1 = image_resized.reshape(1,28, 28, 1) #/ 255

# 反轉顏色，顏色0為白色，與 RGB 色碼不同，它的 0 為黑色
X1 = np.abs(1-X1)

# 預測
predictions = np.argmax(model.predict(X1), axis=-1)
print(predictions)

[9]
